In [8]:
import MySQLdb
import cv2
import requests
import urllib.parse
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import datetime

start_time = datetime.datetime.now()



connection = MySQLdb.connect(host = "127.0.0.1" , user = "root" , password = "zxcv2587", db = 'mltd' , charset = 'utf8') 
cursor = connection.cursor()
cursor.execute('TRUNCATE TABLE `mltd`.`events`')


def get_data(url , date , image):

    req = requests.get(url)
    bs = BeautifulSoup(req.text , 'lxml')
    
    title = bs.select('title')[0].text
    start_date = date[ : 10]
    while(start_date.find('(') != -1):
        start_date=start_date[:-1]
    end_date = date[date.find('～') + 2 : -3]    
    
    pos = title.rfind('～')
    name = title[:pos + 1]        
    
    if title.find('MILLION LIVE WORKING') != -1:
        start_date = date[ : 10]
        end_date = date[date.find('～') + 2 : -3]

        
        
        q = cursor.execute('INSERT INTO `mltd`.`events` (`name` , `start_date` , `end_date` , `image`) VALUES (%s , %s , %s , %s)' ,
            (name  , start_date , end_date , image))        
        
        
    elif title.find('ーツアー') != -1:


        
        string = str(bs)
        pos_A = string.find('イベント限定カード')
        pos_B = string.find('開催日時')
        part = string[pos_A : pos_B]

        bs = BeautifulSoup(part , 'lxml')
        part = bs.select('a')
        
        flag = 0
        
        
        
        
        cards = ''
        
        for i in part:
            if(i['title'] == "SR") or (i['title'] == "R"):
                continue
            if flag == 0:
                flag = 1
                continue
            cards = cards + i['title'] + '\n' #1 2 SR 3 R
            
        q = cursor.execute('INSERT INTO `mltd`.`events` (`name` , `cards` , `start_date` , `end_date` , `image`) VALUES (%s , %s, %s , %s , %s)' ,
            (name , cards[:-1] , start_date , end_date , image))
    
            
            
    elif title.find('ーシアター') != -1:


                
        string = str(bs)
        pos_A = string.find('イベント限定カード')
        pos_B = string.find('開催日時')
        part = string[pos_A : pos_B]

        bs = BeautifulSoup(part , 'lxml')
        part = bs.select('a')
        
        flag = 0;
        
        
        
        
        cards = ''
        for i in part:
            if(i['title'] == "SR") or (i['title'] == "R"):
                continue
            if flag == 0:
                flag = 1
                continue
            cards = cards + i['title'] + '\n' #1 2 SR 3 R        
    
    
        q = cursor.execute('INSERT INTO `mltd`.`events` (`name`, `cards` , `start_date` , `end_date` , `image`) VALUES (%s  , %s , %s , %s ,%s)' ,
            (name , cards[:-1] , start_date , end_date , image))    
    
    
    
    
    elif title.find('COLLECTION') != -1:
                           
                
        
        string = str(bs)
        pos_A = string.find('イベント限定カード')
        pos_B = string.find('開催日時')
        part = string[pos_A : pos_B]

        bs = BeautifulSoup(part , 'lxml')
        part = bs.select('a')
        
        flag = 0;                
        
        
        
        
        
        cards = ''
        for i in part:
            if(i['title'] == "SR") or (i['title'] == "R"):
                continue

            cards = cards + i['title']  + '\n' #1 2 SR 3 R

        q = cursor.execute('INSERT INTO `mltd`.`events` (`name` , `cards` , `start_date` , `end_date` , `image`) VALUES ( %s , %s , %s , %s , %s)' ,
            (name , cards[:-1] , start_date , end_date , image))
            
            
    else:
        
        q = cursor.execute('INSERT INTO `mltd`.`events` (`name` , `start_date` , `end_date` , `image`) VALUES (%s , %s , %s , %s)' ,
            (name , start_date , end_date , image))        
        
    print(name)
        
        
def get_url(hp):
    
    req = requests.get(hp)
    bs = BeautifulSoup(req.text , 'lxml')

    temp = bs.select('.t-line-img')[0]
    td = temp.select('.no-min-width')
    
    
    
    image_list = []
    tag = bs.select('img')
    for line in tag:
        string = str(line)
        if(string.find('width="350"') != -1):
            image_list.append(line['data-original'])
    
    
    
    flag = 1
    flag2 = 1
    count = 0
    url_list = []
    date_list = []
    for i in td:
        
        if(i['data-col'] == '0'):
            if (flag):
                flag = 0
                continue 
            temp = i.select('a')[0]['href']
            if temp not in url_list:
                url_list.append(temp)
                count = count + 1        
        
        elif(i['data-col'] == '1' and i['style'] == 'text-align:center;background:#white !important;'):
            if(i.text.find('2020') != -1 or i.text.find('2019') != -1 or i.text.find('2018') != -1 or i.text.find('2017') != -1):
                date_list.append(i.text)

    for url in url_list:
        get_data(url , date_list[0] , image_list[0])
        date_list.pop(0)
        image_list.pop(0)
        

        
        
        
        
        
homepage_2020 = 'https://imasml-theater-wiki.gamerch.com/%E3%82%A4%E3%83%99%E3%83%B3%E3%83%88%E4%B8%80%E8%A6%A7'
homepage_2019 = 'https://imasml-theater-wiki.gamerch.com/%E3%82%A4%E3%83%99%E3%83%B3%E3%83%88%E4%B8%80%E8%A6%A7%EF%BC%882019.01.01%EF%BD%9E2019.12.31%EF%BC%89'
homepage_2018 = 'https://imasml-theater-wiki.gamerch.com/%E3%82%A4%E3%83%99%E3%83%B3%E3%83%88%E4%B8%80%E8%A6%A7%EF%BC%882018.01.01%EF%BD%9E2018.12.31%EF%BC%89'
homepage_2017 = 'https://imasml-theater-wiki.gamerch.com/%E3%82%A4%E3%83%99%E3%83%B3%E3%83%88%E4%B8%80%E8%A6%A7%EF%BC%882017.07.14%EF%BD%9E2017.12.31%EF%BC%89'
get_url(homepage_2020)
get_url(homepage_2019)
get_url(homepage_2018)
get_url(homepage_2017)

print('')
print(start_time)
print(datetime.datetime.now())

connection.commit()
cursor.close()
connection.close()
print('papara')

【イベント】プラチナスターシアター～アライブファクター～
【ミリシタ】MILLION LIVE WORKING☆～大ヒット！アイドル商品開発部～
【ミリシタ】プラチナスターツアー～絶対的Performer～
【イベント】ミリコレ！～MILLIONLIVE COLLECTION～
【イベント】プラチナスターシアター～Bigバルーン◎～
【ミリシタ】MILLION LIVE WORKING☆～大合奏！アイドルシンフォニー～
【ミリシタ】プラチナスターツアー～クルリウタ～
【イベント】ミリコレ！～MILLIONLIVE COLLECTION～
【ミリシタ】プラチナスターシアター～Helloコンチェルト～
【ミリシタ】超ビーチバレー ～超全国大会編～
【イベント】MILLION LIVE WORKING☆～もっと！輝け！アイドル強化合宿～
【ミリシタ】プラチナスターツインステージ～深淵の蒼、熱焔の緋～
【イベント】ミリコレ！～MILLIONLIVE COLLECTION～
【ミリシタ】プラチナスターシアター～Beat the World!!!～
【ミリシタ】MILLION LIVE WORKING☆～期間限定★アイドルカフェ～
【ミリシタ】プラチナスターツアー～Super Duper～
【イベント】ミリコレ！～MILLIONLIVE COLLECTION～
【イベント】プラチナスターシアター～メメント？モメント♪ルルルルル☆～
【ミリシタ】MILLION LIVE WORKING☆～潜入！スパイシスターズ～
【ミリシタ】プラチナスターツアー～百花は月下に散りぬるを～
【イベント】ミリコレ！～MILLIONLIVE COLLECTION～
【イベント】プラチナスターシアター～俠気乱舞～
【ミリシタ】MILLION LIVE WORKING☆ SPECIAL～夢いっぱい！メルヘンアイドル物語～
【ミリシタ】MILLION LIVE WORKING☆ SPECIAL～夢いっぱい！メルヘンアイドル物語～
【ミリシタ】プラチナスターツアー～Fermata in Rapsodia～
【イベント】ミリコレ！～MILLIONLIVE COLLECTION～
【イベント】プラチナスターシアター～瞳の中のシリウス～
【ミリシタ】MILLION LIVE WORKING☆ ～白熱！アイドル学園頂

In [17]:
import MySQLdb
import cv2
import requests
import urllib.parse
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import datetime

start_time = datetime.datetime.now()






def get_url(hp):
    
    req = requests.get(hp)
    bs = BeautifulSoup(req.text , 'lxml')

    temp = bs.select('.t-line-img')[0]
    td = temp.select('.no-min-width')
    

    tag = bs.select('img')
    for line in tag:
        string = str(line)
        if(string.find('width="350"') != -1):
            print(line['data-original'])
    
    
    
    
    flag = 1
    flag2 = 1
    count = 0
    url_list = []
    date_list = []
    for i in td:
        
        if(i['data-col'] == '0'):
            if (flag):
                flag = 0
                continue 
            temp = i.select('a')[0]['href']
            if temp not in url_list:
                url_list.append(temp)
                count = count + 1        
        
        elif(i['data-col'] == '1' and i['style'] == 'text-align:center;background:#white !important;'):
            if(i.text.find('2020') != -1 or i.text.find('2019') != -1 or i.text.find('2018') != -1 or i.text.find('2017') != -1):
                date_list.append(i.text)

    for url in url_list:
        #get_data(url , date_list[0])
        date_list.pop(0)

        
        
        
        

homepage_2020 = 'https://imasml-theater-wiki.gamerch.com/%E3%82%A4%E3%83%99%E3%83%B3%E3%83%88%E4%B8%80%E8%A6%A7'
homepage_2019 = 'https://imasml-theater-wiki.gamerch.com/%E3%82%A4%E3%83%99%E3%83%B3%E3%83%88%E4%B8%80%E8%A6%A7%EF%BC%882019.01.01%EF%BD%9E2019.12.31%EF%BC%89'
homepage_2018 = 'https://imasml-theater-wiki.gamerch.com/%E3%82%A4%E3%83%99%E3%83%B3%E3%83%88%E4%B8%80%E8%A6%A7%EF%BC%882018.01.01%EF%BD%9E2018.12.31%EF%BC%89'
homepage_2017 = 'https://imasml-theater-wiki.gamerch.com/%E3%82%A4%E3%83%99%E3%83%B3%E3%83%88%E4%B8%80%E8%A6%A7%EF%BC%882017.07.14%EF%BD%9E2017.12.31%EF%BC%89'
get_url(homepage_2020)
# get_url(homepage_2019)
# get_url(homepage_2018)
# get_url(homepage_2017)

print('')
print(start_time)
print(datetime.datetime.now())

https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1590559802.jpg
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1589782278.jpg
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1589178709.jpg
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1588400052.jpg
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1588054762.jpg
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1587277354.jpg
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1586588775.jpg
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1585807663.jpg
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1585667369.jpg
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1585289350.jpg
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1584517952001.jpg
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1583

In [1]:
import MySQLdb
import cv2
import requests
import urllib.parse
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import datetime

start_time = datetime.datetime.now()



connection = MySQLdb.connect(host = "127.0.0.1" , user = "root" , password = "zxcv2587", db = 'mltd' , charset = 'utf8') 
cursor = connection.cursor()
cursor.execute('TRUNCATE TABLE `test`.`events`')


def get_data(url , date , image):

    req = requests.get(url)
    bs = BeautifulSoup(req.text , 'lxml')
    
    title = bs.select('title')[0].text
    start_date = date[ : 10]
    while(start_date.find('(') != -1):
        start_date=start_date[:-1]
    end_date = date[date.find('～') + 2 : -3]    
    
    pos = title.rfind('～')
    name = title[:pos + 1]        
    
    if title.find('MILLION LIVE WORKING') != -1:
        start_date = date[ : 10]
        end_date = date[date.find('～') + 2 : -3]

        
        
        q = cursor.execute('INSERT INTO `mltd`.`events` (`name` , `start_date` , `end_date` , `image`) VALUES (%s , %s , %s , %s)' ,
            (name  , start_date , end_date , image))        
        
        
    elif title.find('ーツアー') != -1:


        
        string = str(bs)
        pos_A = string.find('イベント限定カード')
        pos_B = string.find('開催日時')
        part = string[pos_A : pos_B]

        bs = BeautifulSoup(part , 'lxml')
        part = bs.select('a')
        
        flag = 0
        
        
        
        
        cards = ''
        
        for i in part:
            if(i['title'] == "SR") or (i['title'] == "R"):
                continue
            if flag == 0:
                flag = 1
                continue
            cards = cards + i['title'] + '\n' #1 2 SR 3 R
            
        q = cursor.execute('INSERT INTO `mltd`.`events` (`name` , `cards` , `start_date` , `end_date` , `image`) VALUES (%s , %s, %s , %s , %s)' ,
            (name , cards[:-1] , start_date , end_date , image))
    
            
            
    elif title.find('ーシアター') != -1:


                
        string = str(bs)
        pos_A = string.find('イベント限定カード')
        pos_B = string.find('開催日時')
        part = string[pos_A : pos_B]

        bs = BeautifulSoup(part , 'lxml')
        part = bs.select('a')
        
        flag = 0;
        
        
        
        
        cards = ''
        for i in part:
            if(i['title'] == "SR") or (i['title'] == "R"):
                continue
            if flag == 0:
                flag = 1
                continue
            cards = cards + i['title'] + '\n' #1 2 SR 3 R        
    
    
        q = cursor.execute('INSERT INTO `mltd`.`events` (`name`, `cards` , `start_date` , `end_date` , `image`) VALUES (%s  , %s , %s , %s ,%s)' ,
            (name , cards[:-1] , start_date , end_date , image))    
    
    
    
    
    elif title.find('COLLECTION') != -1:
                           
                
        
        string = str(bs)
        pos_A = string.find('イベント限定カード')
        pos_B = string.find('開催日時')
        part = string[pos_A : pos_B]

        bs = BeautifulSoup(part , 'lxml')
        part = bs.select('a')
        
        flag = 0;                
        
        
        
        
        
        cards = ''
        for i in part:
            if(i['title'] == "SR") or (i['title'] == "R"):
                continue

            cards = cards + i['title']  + '\n' #1 2 SR 3 R

        q = cursor.execute('INSERT INTO `mltd`.`events` (`name` , `cards` , `start_date` , `end_date` , `image`) VALUES ( %s , %s , %s , %s , %s)' ,
            (name , cards[:-1] , start_date , end_date , image))
            
            
    else:
        
        q = cursor.execute('INSERT INTO `mltd`.`events` (`name` , `start_date` , `end_date` , `image`) VALUES (%s , %s , %s , %s)' ,
            (name , start_date , end_date , image))        
        
    print(name)
        
        
def get_url(hp):
    
    req = requests.get(hp)
    bs = BeautifulSoup(req.text , 'lxml')

    temp = bs.select('.t-line-img')[0]
    td = temp.select('.no-min-width')
    
    
    
    image_list = []
    tag = bs.select('img')
    for line in tag:
        string = str(line)
        if(string.find('width="350"') != -1):
            image_list.append(line['data-original'])
    
    
    
    flag = 1
    flag2 = 1
    count = 0
    url_list = []
    date_list = []
    for i in td:
        
        if(i['data-col'] == '0'):
            if (flag):
                flag = 0
                continue 
            temp = i.select('a')[0]['href']
            if temp not in url_list:
                url_list.append(temp)
                count = count + 1        
        
        elif(i['data-col'] == '1' and i['style'] == 'text-align:center;background:#white !important;'):
            if(i.text.find('2020') != -1 or i.text.find('2019') != -1 or i.text.find('2018') != -1 or i.text.find('2017') != -1):
                date_list.append(i.text)

    for url in url_list:
        get_data(url , date_list[0] , image_list[0])
        date_list.pop(0)
        image_list.pop(0)
        

        
        
        
        
        
homepage_2020 = 'https://imasml-theater-wiki.gamerch.com/%E3%82%A4%E3%83%99%E3%83%B3%E3%83%88%E4%B8%80%E8%A6%A7'
get_url(homepage_2020)


print('')
print(start_time)
print(datetime.datetime.now())

connection.commit()
cursor.close()
connection.close()
print('papara')

IntegrityError: (1364, "Field 'name_tw' doesn't have a default value")

In [91]:
import MySQLdb
import cv2
import requests
import urllib.parse
from urllib.parse import unquote

from bs4 import BeautifulSoup
from urllib.parse import urljoin
import datetime

start_time = datetime.datetime.now()



connection = MySQLdb.connect(host = "127.0.0.1" , user = "root" , password = "zxcv2587", db = 'mltd' , charset = 'utf8') 
cursor = connection.cursor()
cursor.execute('TRUNCATE TABLE `test`.`events_2019`')



    

        
        
def get_url(hp):
    
    req = requests.get(hp)
    bs = BeautifulSoup(req.text , 'lxml')

    table = bs.select('tbody')[0]
    data = table.select('td')
    flag = 0
    count = 300
    for i in data:
        if(i['data-col'] == '1' and flag == 1):
            date = i.text
            start_date = date[:date.find(')')-2]
            date = date[date.find(')'):]
            end_date = date[date.find('20'):date.rfind(')')-2]
            if(len(end_date) == 0):
                end_date = start_date

            
        if(i['data-col'] == '0' ):
            img = i.select('img')
            if(len(img) == 0):
                flag = 1
                temp = i.select('a')[0]
                ori_url = temp['href'] #url
                
                ori_url = urllib.parse.unquote(ori_url)    #utf-8
                name_jp = temp.text #name
            else:
                count = count + 1
                flag = 0
                image = img[0]['data-original'] #image
                
                print(name_jp)
                print(start_date)
                print(end_date)                
                print(image)
                print(ori_url)
                
                
                
                q = cursor.execute('INSERT into `test`.`events_2019`(`id` , `name_jp` , `name_tw` , `start_date` , `end_date` , `image` , `ori_url`) VALUES(%s , %s, %s , %s , %s , %s , %s)' ,
                    (count , name_jp , name_jp , start_date , end_date , image , ori_url))
    
    
        
    
    

        
        
        
        
        
homepage_2019 = 'https://imasml-theater-wiki.gamerch.com/%E3%82%A4%E3%83%99%E3%83%B3%E3%83%88%E4%B8%80%E8%A6%A7%EF%BC%882019.01.01%EF%BD%9E2019.12.31%EF%BC%89'
get_url(homepage_2019)



print('')
print(start_time)
print(datetime.datetime.now())

connection.commit()
cursor.close()
connection.close()
print('papara')

MILLION LIVE WORKING☆ SPECIAL
～夢いっぱい！メルヘンアイドル物語～
2019/12/26
2020/01/01
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1577344916.jpg
https://imasml-theater-wiki.gamerch.com/MILLION LIVE WORKING☆ SPECIAL～夢いっぱい！メルヘンアイドル物語～
プラチナスターツアー～Fermata in Rapsodia～
2019/12/17
2019/12/24
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1576563055.jpg
https://imasml-theater-wiki.gamerch.com/【イベント】プラチナスターツアー～Fermata in Rapsodia～
ミリコレ！～MILLIONLIVE COLLECTION～
2019/12/10
2019/12/15
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1575957750.jpg
https://imasml-theater-wiki.gamerch.com/【イベント】ミリコレ！～MILLIONLIVE COLLECTION～ 201912
プラチナスターシアター～瞳の中のシリウス～
2019/12/01
2019/12/08
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wiki/1575200497.jpg
https://imasml-theater-wiki.gamerch.com/【イベント】プラチナスターシアター～瞳の中のシリウス～
MILLION LIVE WORKING☆
～白熱！アイドル学園頂上決戦～
2019/11/27
2019/11/30
https://cdn.img-conv.gamerch.com/img.gamerch.com/imasml-theater-wi